# Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from collections import defaultdict
import text_hammer as th
import missingno as msno
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from transformers import AutoTokenizer,TFBertModel
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import gensim
from nltk.data import find
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras import Sequential, Input, optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
from keras.callbacks import ReduceLROnPlateau
from keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import plot_model
import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments
from transformers import TFBertModel, Trainer
from tokenizers import Tokenizer
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import missingno as msno
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchmetrics
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning import Callback, LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from tqdm.notebook import tqdm
import json
from sklearn.metrics import precision_recall_fscore_support
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import transformers
from transformers import BertModel, BertTokenizerFast
from transformers import DistilBertModel, DistilBertTokenizerFast, AdamW
from wordcloud import WordCloud
import os
import warnings
warnings.filterwarnings('ignore')

# Realizando a leitura do dataframe de treino

In [ ]:
nlp_train = pd.read_csv('../input/nlp-getting-started/train.csv')
nlp_train.head(2)

In [ ]:
nlp_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
nlp_test.head(2)

# Inserindo o esquema de cores
O esquema de cores na linguagem de programação Python se estabelece a partir do uso a ser totalmente utilizado em que o notebook será analisado ou treinado, muitas das vezes bastante usado em visualização de dados.

In [ ]:
custom_colors = ['#000000', '#E31E33', '#4A53E1', '#F5AD02', '#94D5EA', '#F6F8F7']
custom_palette = sns.set_palette(sns.color_palette(custom_colors))
sns.palplot(sns.color_palette(custom_colors), size=1)
plt.tick_params(axis='both', labelsize=0, length=0)

# EDA

In [ ]:
# using dataprep package - EDA
from dataprep.eda import plot
plot(nlp_train)

In [ ]:
# Analyse Correlations
from dataprep.eda import plot_correlation
plot_correlation(nlp_train)

In [ ]:
# Buscando a contagem dos destinos de na coluna do dataframe Target.
plt.title('Contagem dos destinos Target (0 e 1)')
sns.countplot(nlp_train['target'])
plt.show()

In [ ]:
# Buscando a contagem de cada desastre acontecido dentro do twitter.
plt.figure(figsize = (15, 12))
ax = plt.axes()
ax.set_facecolor('white')
ax = sns.countplot(x = 'target', data = nlp_train, palette = [custom_colors[2], custom_colors[1]], edgecolor = 'white', linewidth = 1.2)
plt.title('Contagem de desastres', fontsize = 25)
plt.xlabel('Desastre', fontsize = 20)
plt.ylabel('Contagem do desastre', fontsize = 20)
ax.xaxis.set_tick_params(labelsize = 15)
ax.yaxis.set_tick_params(labelsize = 15)
bbox_args = dict(boxstyle = 'round', fc = '0.9')
for p in ax.patches:
        ax.annotate('{:.0f} = {:.2f}%'.format(p.get_height(), (p.get_height() / len(nlp_train['target'])) * 100), (p.get_x() + 0.25, p.get_height() + 60), 
                   color = 'black',
                   bbox = bbox_args,
                   fontsize = 15)
plt.show()

In [ ]:
# Buscando a contagens em cada coluna do dataframe.
msno.bar(nlp_train, color=(0, 0, 0), sort="ascending", figsize=(15, 10))
plt.show()

In [ ]:
# Palavras mais repetidas no dataframe de Treino e Teste.
plt.figure(figsize=(15,40))
print(f'Buscando as Palavras-chave únicas no dataframe de treino = {len(nlp_train.keyword.unique())}')
print(f'Buscando as Palavras-chave únicas no dataframe de teste=  {len(nlp_test.keyword.unique())}')
sns.countplot(y=nlp_train['keyword'], color=(0,0,1), label='Treino')
sns.countplot(y=nlp_test['keyword'], color=(0,0,0), label='Teste')
plt.legend()
plt.show()

In [ ]:
# Deixando a contagem de palavras repetidas com uma visualização de dados mais limpa e mais bem visual.
plt.figure(figsize=(15,100))
sns.countplot(data=nlp_train, y='keyword', hue='target')
plt.show()

In [ ]:
# Verificando as localizações dos desastres acontecidos.
nlp_train['location'].value_counts()[:10]

In [ ]:
plt.figure(figsize=(15, 13))
ax = plt.axes()
ax.set_facecolor('white')
ax = ((nlp_train.location.value_counts())[:10]).plot(kind='bar', color=custom_colors[3], linewidth=2, edgecolor='white')
plt.title('Contagem das localizações', fontsize=30)
plt.xlabel('Localização', fontsize=25)
plt.ylabel('Número da contagem', fontsize=25)
ax.xaxis.set_tick_params(labelsize=15, rotation=30)
ax.yaxis.set_tick_params(labelsize=15)
bbox_args = dict(boxstyle='round', fc='0.9')
for p in ax.patches:
        ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x() + 0.15, p.get_height() + 2), bbox=bbox_args, color=custom_colors[0], fontsize=10)

# Nesse treinamento estamos transformando os dados e classificando seus ajustes

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())
])

In [ ]:
text_clf.fit(nlp_train['text'], nlp_train['target'])

# Nesse treinamento usamos TF-IDF com a Regressão Logística

In [ ]:
X = nlp_train['text']
y = nlp_train['target']

In [ ]:
skf = StratifiedKFold(n_splits=5)
train_average_score = 0
validation_average_score = 0
validation_oof_predictions = np.zeros((len(X)))

for fold_n, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    model = Pipeline([
        ('Encoder', TfidfVectorizer(max_features=None)),
        ('Clf', LogisticRegression(penalty='l2', C=1, solver='liblinear'))
    ])
    
    model.fit(X_train, y_train)
    
    train_predictions = model.predict_proba(X_train)[:, 1]
    validation_predictions = model.predict_proba(X_test)[:, 1]
    
    train_score = roc_auc_score(y_train, train_predictions)
    validation_score = roc_auc_score(y_test, validation_predictions)
    
    train_average_score += train_score / 5
    validation_average_score += validation_score / 5
    validation_oof_predictions[test_idx,] = (validation_predictions > 0.5).astype(int)
    
print(f'Fold: {fold_n}, train auc: {train_score:.3f}, validation auc: {validation_score:.3f}')
print(f'Train average: {train_average_score:.3f}, validation average: {validation_average_score:.3f}')
print(f'OOF Accuracy Score: {accuracy_score(y, validation_oof_predictions)}')

# Prever e salvar para enviar

In [ ]:
nlp_test['target'] = text_clf.predict(nlp_test['text'])

In [ ]:
nlp_test[['id', 'target']].to_csv('submission.csv', index=False)